# layers

> Pure configuration objects for describing categorical raster layers.

In [1]:
#| default_exp layers

In [2]:
#| hide
from nbdev.showdoc import *

## The Core Abstraction

A `CategoricalLayer` describes *where* categorical raster data lives and *how* to access it — nothing more.

It answers:
- Where is the data? (`asset_id`)
- How are yearly bands named? (`band_pattern`)
- What resolution? (`scale`)

Optionally:
- What do the class values mean? (`class_map`)
- What colors for visualization? (`palette`)

This object contains **no Earth Engine calls**, **no geometry**, **no statistics**. Just descriptors.

In [3]:
#| export
from dataclasses import dataclass, field
from typing import Optional, Dict

In [4]:
#| export
@dataclass
class CategoricalLayer:
    """A descriptor for a categorical raster layer with annual bands.
    
    This is a pure configuration object — no GEE calls, no logic.
    
    Attributes:
        asset_id: GEE asset path (e.g., 'projects/mapbiomas-public/assets/...')
        band_pattern: Pattern for yearly bands with {} placeholder (e.g., 'classification_{}')
        scale: Pixel resolution in meters (default 30)
        class_map: Optional mapping of class values to names
        palette: Optional mapping of class values to hex colors
    """
    asset_id: str
    band_pattern: str
    scale: int = 30
    class_map: Optional[Dict[int, str]] = None
    palette: Optional[Dict[int, str]] = None
    
    def band_name(self, year: int) -> str:
        """Get the band name for a specific year."""
        return self.band_pattern.format(year)
    
    def class_name(self, class_value: int) -> Optional[str]:
        """Get the human-readable name for a class value."""
        if self.class_map is None:
            return None
        return self.class_map.get(class_value)
    
    def class_color(self, class_value: int) -> Optional[str]:
        """Get the hex color for a class value."""
        if self.palette is None:
            return None
        return self.palette.get(class_value)

## Example: Creating a Layer

Anyone can create a `CategoricalLayer` for any GEE asset:

In [5]:
# A minimal layer — just location and band pattern
my_layer = CategoricalLayer(
    asset_id="users/foo/my_classification",
    band_pattern="class_{}"
)

print(my_layer)
print(f"Band for 2020: {my_layer.band_name(2020)}")

CategoricalLayer(asset_id='users/foo/my_classification', band_pattern='class_{}', scale=30, class_map=None, palette=None)
Band for 2020: class_2020


In [6]:
# A layer with metadata
layer_with_metadata = CategoricalLayer(
    asset_id="users/foo/my_classification",
    band_pattern="class_{}",
    scale=30,
    class_map={
        0: "No Data",
        1: "Forest",
        2: "Non-Forest",
        3: "Water"
    },
    palette={
        1: "#228B22",
        2: "#DEB887", 
        3: "#4169E1"
    }
)

print(f"Class 1 is: {layer_with_metadata.class_name(1)}")
print(f"Class 1 color: {layer_with_metadata.class_color(1)}")

Class 1 is: Forest
Class 1 color: #228B22


The `CategoricalLayer` is intentionally simple. Dataset-specific presets (like MapBiomas) are defined in `gee_polygons.datasets` modules.

In [7]:
#| hide
import nbdev; nbdev.nbdev_export()